In [ ]:
#| default_exp weather_openapi
"""
Auto‑generate MCP tools for Open‑Meteo (glue‑free).

* Downloads the canonical OpenAPI 3.0 file published by Open‑Meteo.
* Builds a FastMCP sub‑server from the spec.
* Mounts that sub‑server into our main `MCP` so the LLM can call
  any endpoint under `/v1/forecast`, `/v1/marine`, `/v1/air-quality`, etc.
"""
from __future__ import annotations

import asyncio
from typing import Any

import httpx, yaml
from fastmcp import FastMCP  # FastMCP ≥ 2.0



# -----------------------------------------------------------------------------
# Open‑Meteo OpenAPI spec
# -----------------------------------------------------------------------------
SPEC_URL = (
    "https://raw.githubusercontent.com/open-meteo/"
    "open-meteo/main/openapi.yml"
)

# -----------------------------------------------------------------------------
# Helpers
# -----------------------------------------------------------------------------
async def _load_openapi_spec(url: str) -> dict[str, Any]:
    """Download and parse the YAML OpenAPI file once at startup."""
    async with httpx.AsyncClient(timeout=10) as cx:
        res = await cx.get(url)
    res.raise_for_status()
    return yaml.safe_load(res.text)


async def mount_open_meteo(mcp: FastMCP) -> None:
    """Load the Open‑Meteo spec ➜ generate a FastMCP sub‑server ➜ mount it."""
    spec = await _load_openapi_spec(SPEC_URL)

    # The spec already contains the upstream base URL; FastMCP still wants a client.
    client = httpx.AsyncClient(base_url="https://api.open-meteo.com")

    weather_srv = FastMCP.from_openapi(
        openapi_spec=spec,
        client=client,
        name="open-meteo",
    )

    
    mcp.mount(weather_srv)


